In [1]:
%run imports.py
%matplotlib inline
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [2]:
import torchvision.models as models
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [3]:
EXP = "exp2"

In [4]:
if not os.path.isdir("exps"):
    os.mkdir("exps")
if not os.path.isdir("exps/" + EXP):
    os.mkdir("exps/" + EXP)

In [5]:
%tensorboard --logdir exps/{EXP}/logs

Reusing TensorBoard on port 6006 (pid 26716), started 2:36:13 ago. (Use '!kill 26716' to kill it.)

In [6]:
input_batch = torch.ones((64, 3, 32, 32))
writer = SummaryWriter(log_dir="exps/" + EXP + "/logs/mobilenet_graph")
writer.add_graph(mobilenet, input_batch)
writer.close()

In [104]:
with open("exps/default_config.yaml", "r") as f:
    default_config = yaml.load(f, Loader=yaml.SafeLoader)
exp_config = dict()
if os.path.isfile("exps/" + EXP + "/config.yaml"):
    with open("exps/" + EXP + "/config.yaml", "r") as f:
        exp_config = yaml.load(f, Loader=yaml.SafeLoader)
config = dict()
config.update(default_config)
config.update(exp_config)
print(config)

{'dataset': {'frames_per_clip': 10, 'step_between_clips': 10, 'batch_size': 32, 'transform': {'Normalize': {'mean': 0.449, 'std': 0.226}, 'HorizontalFlip': True, 'Cutout': True}, 'transformOF': {'Normalize': None, 'HorizontalFlip': True, 'Cutout': False}}, 'model': {'phase1': {'epochs': 10, 'lr': 0.005, 'step_size': 5, 'gamma': 0.5}, 'phase2': {'epochs': 30, 'lr': 5e-05, 'step_size': 10, 'gamma': 0.5}}}


In [8]:
tansform_param = config["dataset"]["transform"]
albu_aug = [alb.Resize(256, 256, interpolation=cv2.INTER_CUBIC, always_apply=True),
            alb.RandomCrop(244, 244, always_apply=True)]
if tansform_param["HorizontalFlip"]:
    albu_aug.append(aat.HorizontalFlip())
if tansform_param["Cutout"]:
    albu_aug.append(aat.Cutout(2, 10, 10))
tensor_aug = [AlbuWrapperNumpy(alb.Compose(albu_aug)),transforms.ToTensor()]
if tansform_param["Normalize"]:
    norm = tansform_param["Normalize"]
    tensor_aug.append(transforms.Normalize(norm["mean"],norm["std"]))

In [9]:
%run preprocessing/CustomTransform.py
tansformOF_param = config["dataset"]["transformOF"]
albu_augOF = [alb.Resize(256, 256, interpolation=cv2.INTER_CUBIC, always_apply=True),
            alb.RandomCrop(244, 244, always_apply=True)]
if tansformOF_param["HorizontalFlip"]:
    albu_augOF.append(HorizontalFlipOF())
if tansformOF_param["Cutout"]:
    albu_augOF.append(aat.Cutout(2, 10, 10))
tensor_augOF = [AlbuWrapperNumpy(alb.Compose(albu_aug)),transforms.ToTensor()]
if tansformOF_param["Normalize"]:
    norm = tansformOF_param["Normalize"]
    tensor_augOF.append(transforms.Normalize(norm["mean"],norm["std"]))

In [10]:
alb_rescale = alb.Resize(244, 244, always_apply=True)


test_transforms = transforms.Compose(
  [AlbuWrapperNumpy(alb_rescale), *tensor_aug])
train_transforms = transforms.Compose(
  [AlbuWrapperNumpy(alb.Compose(albu_aug)), *tensor_aug])


test_transformsOF = transforms.Compose(
  [AlbuWrapperNumpy(alb_rescale), *tensor_augOF])
train_transformsOF = transforms.Compose(
  [AlbuWrapperNumpy(alb.Compose(albu_augOF)), *tensor_augOF])

In [53]:
%run preprocessing/FallDetectionDataset.py

frames_per_clip = config["dataset"]["frames_per_clip"]
step_between_clips = config["dataset"]["step_between_clips"]
batch_size = config["dataset"]["batch_size"]


train_dataset = FallDetectionDataset(train=True, step_between_clips=step_between_clips,
                                     frames_per_clip=frames_per_clip, optical_flow=False,
                                     verbose=False, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, pin_memory = True,
                                          shuffle=True, num_workers=0)

test_dataset = FallDetectionDataset(train=False, step_between_clips=step_between_clips,
                                     frames_per_clip=frames_per_clip, optical_flow=False,
                                     verbose=False, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, pin_memory = True,
                                          shuffle=True, num_workers=0)


train_datasetOF = FallDetectionDataset(train=True, step_between_clips=step_between_clips,
                                     frames_per_clip=frames_per_clip,
                                     verbose=False, transform=train_transformsOF)
train_loaderOF = torch.utils.data.DataLoader(train_datasetOF, batch_size=batch_size, pin_memory = True,
                                          shuffle=True, num_workers=0)

test_datasetOF = FallDetectionDataset(train=False, step_between_clips=step_between_clips,
                                     frames_per_clip=frames_per_clip,
                                     verbose=False, transform=test_transformsOF)
test_loaderOF = torch.utils.data.DataLoader(test_datasetOF, batch_size=batch_size, pin_memory = True,
                                              shuffle=True, num_workers=0)

targets = np.asarray([target for _, target in train_dataset])
(unique, counts) = np.unique(targets, return_counts=True)
weights = torch.from_numpy((counts.min()/counts).astype(np.float32))
print("Clases: ", unique, ", with frequency: ", counts)
print("Weights: ", weights)

loss_function = torch.nn.CrossEntropyLoss(weight=weights).cuda()if torch.cuda.is_available() else torch.nn.CrossEntropyLoss(weight=weights)

Clases:  [0 1 2 3 4 5] , with frequency:  [ 64 754 370 480  62  73]
Weights:  tensor([0.9688, 0.0822, 0.1676, 0.1292, 1.0000, 0.8493])


In [35]:
print("Train batches: ", len(train_loader), "images, and ", len(train_loaderOF))
print("Train shape: ", next(iter(train_loader))[0].shape, "images, and ", next(iter(train_loaderOF))[0].shape, "flows")
print("Test batches: ", len(test_loader), "images, and ", len(test_loaderOF))
print("Test shape: ", next(iter(test_loader))[0].shape, "images, and ", next(iter(test_loaderOF))[0].shape, "flows")

Train batches:  57 images, and  57
Train shape:  torch.Size([32, 3, 244, 244]) images, and  torch.Size([32, 20, 244, 244]) flows
Test batches:  11 images, and  11
Test shape:  torch.Size([32, 3, 244, 244]) images, and  torch.Size([32, 20, 244, 244]) flows


In [36]:
def train(model, train_loader, optimizer, loss_function, epoch, writer=None, device="cpu"):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_loss /= len(train_loader.dataset)
    if writer:
        writer.add_scalar("loss/train", train_loss, global_step=epoch)
    return train_loss

In [37]:
def test(model, test_loader, loss_function, epoch, writer=None, device="cpu"):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    if writer:
        writer.add_scalar("loss/test", test_loss, global_step=epoch)
        writer.add_scalar("accuracy", accuracy, global_step=epoch)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    return test_loss

In [38]:
def train_model(model, train_loader, test_loader, lr=0.1, epochs=5, step_size=5, gamma=0.1,
                start_epoch=0, device_name="cpu", save_path=None, model_name=""):
    device = torch.device(device_name)
    model = model.to(device)
    writer = SummaryWriter(log_dir=f"exps/{EXP}/logs/{model_name}")
    optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0)
    scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size, gamma=gamma)
    loss = 0
    train_loss = 200
    no_advance = 0
    print("Started training")
    for epoch in range(start_epoch, start_epoch + epochs):
        new_train_loss = train(model, train_loader, optim, loss_function, epoch, writer, device)
        if save_path and (new_train_loss < train_loss):
            torch.save(model.state_dict(), os.path.join(PATH, f"{model_name}_best.pth"))
        train_loss = new_train_loss
        
        new_loss = test(model, test_loader, loss_function, epoch, writer, device)
        no_advance = no_advance + 1 if new_loss - loss > 0 else 0
        if no_advance >= 10:
            break
        loss = new_loss
        scheduler.step()
        print("Current LR: ", scheduler.get_lr())
    writer.close()
    return model

In [39]:
# average input channels of conv layer and return new model
def replace_conv(model, inplanes):
    conv_weights = list(model.parameters())[0].clone().detach().numpy()
    new_conv_weights = conv_weights.mean(axis=1)
    new_conv_weights = np.repeat(new_conv_weights[:, np.newaxis, :, :], inplanes, axis=1)
    new_conv = torch.nn.Conv2d(inplanes, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    new_conv.weight = torch.nn.Parameter(torch.from_numpy(new_conv_weights))
    model._modules['features'][0][0] = new_conv
    return model

def replace_classifier(model, outplanes):
    classifier = torch.nn.Linear(in_features=1280, out_features=outplanes, bias=True)
    model._modules['classifier'][1] = classifier
    return model

def get_model(freeze=True, inplanes=10, outplanes=2, pretrained=True,
              start_model=None, model_name="model"):
    model = start_model
    if start_model is None:
        model = models.mobilenet_v2(pretrained=pretrained)

    if inplanes != model._modules["features"][0][0].in_channels:
        model = replace_conv(model, inplanes)
        
    if outplanes != model._modules["classifier"][1].out_features:
        model = replace_classifier(model, outplanes)
        
    for p in model.features.parameters():
        p.requires_grad = (not freeze)
    input_batch = torch.ones((batch_size, inplanes, 32, 32))
    writer = SummaryWriter(log_dir=f"exps/{EXP}/logs/{model_name}")
    writer.add_graph(model, input_batch)
    return model

def toggle_freeze(model, freeze):
    for p in model.features.parameters():
        p.requires_grad = (not freeze)
    return model

In [85]:
# experiment with lr

first_batch = torch.utils.data.Subset(train_dataset, range(0, 512))
exp_train_loader = torch.utils.data.DataLoader(first_batch, batch_size=32, shuffle=True)
first_batch_test = torch.utils.data.Subset(test_dataset, range(0, 64))
exp_test_loader = torch.utils.data.DataLoader(first_batch_test, batch_size=32, shuffle=False)


PATH = f"exps/{EXP}/"
MODEL_NAME="images_model"

model_config = config["model"]
p1_config = model_config["phase1"]
p2_config = model_config["phase2"]

model = get_model(freeze=True, inplanes=3, outplanes=6, model_name=MODEL_NAME)
%time train_model(model, train_loader=exp_train_loader, test_loader=exp_test_loader, lr=0.005, epochs=2, step_size=5, gamma=0.5, device_name="cuda", model_name=MODEL_NAME, save_path=PATH)
toggle_freeze(model, False)
%time train_model(model, train_loader=exp_train_loader, test_loader=exp_test_loader, lr=0.00005, epochs=2, step_size=5, gamma=0.5, device_name="cuda", model_name=MODEL_NAME, save_path=PATH)
print("finished exp")

Started training
Train Epoch: 0 [0/512 (0%)]	Loss: 1.760324
Train Epoch: 0 [160/512 (31%)]	Loss: 1.871633
Train Epoch: 0 [320/512 (62%)]	Loss: 3.250012
Train Epoch: 0 [480/512 (94%)]	Loss: 2.931759

Test set: Average loss: 0.0949, Accuracy: 26/64 (41%)

Current LR:  [0.005]
Train Epoch: 1 [0/512 (0%)]	Loss: 1.950399
Train Epoch: 1 [160/512 (31%)]	Loss: 1.928668
Train Epoch: 1 [320/512 (62%)]	Loss: 1.400463
Train Epoch: 1 [480/512 (94%)]	Loss: 1.243181

Test set: Average loss: 0.1269, Accuracy: 22/64 (34%)

Current LR:  [0.005]
Wall time: 11.3 s
Started training
Train Epoch: 0 [0/512 (0%)]	Loss: 1.156298
Train Epoch: 0 [160/512 (31%)]	Loss: 0.865721
Train Epoch: 0 [320/512 (62%)]	Loss: 1.043439
Train Epoch: 0 [480/512 (94%)]	Loss: 1.123975

Test set: Average loss: 0.1109, Accuracy: 29/64 (45%)

Current LR:  [5e-05]
Train Epoch: 1 [0/512 (0%)]	Loss: 1.117422
Train Epoch: 1 [160/512 (31%)]	Loss: 0.836988
Train Epoch: 1 [320/512 (62%)]	Loss: 0.754429
Train Epoch: 1 [480/512 (94%)]	Loss: 0.

In [76]:
# look at predictions

# model = get_model(inplanes=3, outplanes=6, model_name=MODEL_NAME)
device = torch.device("cuda")
model = model.to(device)
with torch.no_grad():
    correct = 0
    for data, target in exp_test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss = loss_function(output, target).sum().item()
        print(test_loss/len(exp_test_loader.dataset))
#         print(output)
        pred = output.argmax(dim=1, keepdim=True)
        target_view = target.view_as(pred)
        correct += pred.eq(target.view_as(pred)).sum().item()
        for index, (pred_label, true_label) in enumerate(zip(pred, target_view)):
            print(pred_label, true_label)
    print(correct, len(exp_train_loader.dataset))
# model = None
torch.cuda.empty_cache()

0.7437428832054138
tensor([1], device='cuda:0') tensor([0], device='cuda:0')
tensor([1], device='cuda:0') tensor([0], device='cuda:0')
tensor([4], device='cuda:0') tensor([0], device='cuda:0')
tensor([4], device='cuda:0') tensor([0], device='cuda:0')
tensor([1], device='cuda:0') tensor([0], device='cuda:0')
tensor([4], device='cuda:0') tensor([1], device='cuda:0')
tensor([1], device='cuda:0') tensor([1], device='cuda:0')
tensor([1], device='cuda:0') tensor([1], device='cuda:0')
tensor([1], device='cuda:0') tensor([1], device='cuda:0')
tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([1], device='cuda:0') tensor([4], device='cuda:0')
tensor([1], device='cuda:0') tensor([4], device='cuda:0')
tensor([1], device='cuda:0') tensor([2], device='cuda

In [74]:
print(model._modules["classifier"][1].weight.grad.min())
print(model._modules["classifier"][1].weight.grad.max())
print(model._modules["classifier"][1].weight.grad.median())
print(model._modules["classifier"][1].weight.grad.std())


print((model._modules["classifier"][1].bias / model._modules["classifier"][1].bias.grad).abs().min())
print((model._modules["classifier"][1].bias / model._modules["classifier"][1].bias.grad).abs().median())
print((model._modules["classifier"][1].bias / model._modules["classifier"][1].bias.grad).abs().max())

print((model._modules["classifier"][1].weight / model._modules["classifier"][1].weight.grad).abs().min())
print((model._modules["classifier"][1].weight / model._modules["classifier"][1].weight.grad).abs().median())
print((model._modules["classifier"][1].weight / model._modules["classifier"][1].weight.grad).abs().max())

tensor(-0.1832, device='cuda:0')
tensor(0.1367, device='cuda:0')
tensor(0.0005, device='cuda:0')
tensor(0.0380, device='cuda:0')
tensor(0.0008, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.1748, device='cuda:0', grad_fn=<MedianBackward0>)
tensor(1.6818, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0001, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.6188, device='cuda:0', grad_fn=<MedianBackward0>)
tensor(2358.4880, device='cuda:0', grad_fn=<MaxBackward1>)


In [65]:
# clear GPU usage
model = None
torch.cuda.empty_cache()

In [ ]:
PATH = f"exps/{EXP}/"
MODEL_NAME="images_model"
MODEL_FILE=f"{MODEL_NAME}_best.pth"
# MODEL_FILE=f"{MODEL_NAME}_final.pth"

model_config = config["model"]
p1_config = model_config["phase1"]
p2_config = model_config["phase2"]

model = get_model(inplanes=3, outplanes=6, model_name=MODEL_NAME)
if os.path.isfile(os.path.join(PATH, MODEL_FILE)):
    model.load_state_dict(torch.load(os.path.join(PATH, MODEL_FILE)))

%time train_model(model, train_loader=train_loader, test_loader=test_loader, lr=p1_config["lr"], epochs=p1_config["epochs"], step_size=p1_config["step_size"], gamma=p1_config["gamma"], device_name="cuda", model_name=MODEL_NAME, save_path=PATH)

model = toggle_freeze(model, False)
%time train_model(model, train_loader=train_loader, test_loader=test_loader, start_epoch=p1_config["epochs"], lr=p2_config["lr"], epochs=p2_config["epochs"],  step_size=p2_config["step_size"], gamma=p2_config["gamma"], device_name="cuda", model_name=MODEL_NAME, save_path=PATH)

torch.save(model.state_dict(), os.path.join(PATH, MODEL_FILE))
writer.close()
model = None
torch.cuda.empty_cache()

Started training
Train Epoch: 10 [0/1803 (0%)]	Loss: 0.949408
Train Epoch: 10 [160/1803 (9%)]	Loss: 0.530955
Train Epoch: 10 [320/1803 (18%)]	Loss: 1.228721
Train Epoch: 10 [480/1803 (26%)]	Loss: 1.328973
Train Epoch: 10 [640/1803 (35%)]	Loss: 0.940009
Train Epoch: 10 [800/1803 (44%)]	Loss: 0.624783
Train Epoch: 10 [960/1803 (53%)]	Loss: 0.847244
Train Epoch: 10 [1120/1803 (61%)]	Loss: 0.769361
Train Epoch: 10 [1280/1803 (70%)]	Loss: 0.912148
Train Epoch: 10 [1440/1803 (79%)]	Loss: 1.212388
Train Epoch: 10 [1600/1803 (88%)]	Loss: 0.704617
Train Epoch: 10 [1760/1803 (96%)]	Loss: 0.803320

Test set: Average loss: 0.1026, Accuracy: 170/333 (51%)

Current LR:  [5e-05]
Train Epoch: 11 [0/1803 (0%)]	Loss: 0.986056


In [ ]:
MODEL_NAME="of_model"
MODEL_FILE=f"{MODEL_NAME}_best.pth"
# MODEL_FILE=f"{MODEL_NAME}_final.pth"

model_config = config["model"]
p1_config = model_config["phase1"]
p2_config = model_config["phase2"]

model = get_model(inplanes=frames_per_clip*2, outplanes=6, model_name=MODEL_NAME)
if os.path.isfile(os.path.join(PATH, MODEL_FILE)):
    model.load_state_dict(torch.load(os.path.join(PATH, MODEL_FILE)))

%time train_model(model, train_loader=train_loaderOF, test_loader=test_loaderOF, lr=p1_config["lr"], epochs=p1_config["epochs"], step_size=p1_config["step_size"], gamma=p1_config["gamma"], device_name="cuda", model_name=MODEL_NAME, save_path=PATH)

model = toggle_freeze(model, freeze)
%time train_model(model, train_loader=train_loaderOF, test_loader=test_loaderOF, start_epoch=p1_config["epochs"], lr=p2_config["lr"], epochs=p2_config["epochs"], step_size=p2_config["step_size"], gamma=p2_config["gamma"], device_name="cuda", model_name=MODEL_NAME, save_path=PATH)

torch.save(model.state_dict(), os.path.join(PATH, MODEL_FILE))
writer.close()
model = None
torch.cuda.empty_cache()

In [103]:
def test_combined(images_model, of_model, image_data_loader, of_data_loader):
    device = torch.device("cuda")
    images_model = images_model.to(device)
    of_model = of_model.to(device)
    with torch.no_grad():
        correct = 0
        test_loss = 0
        for (img, target), (of, _) in zip(image_data_loader, of_data_loader):
            img = img.to(device)
            of = of.to(device)
            target = target.to(device)
            imgs_output = images_model(img)
            of_output = of_model(of)
            output = 2 * of_output + imgs_output
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)
            target_view = target.view_as(pred)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(of_data_loader)
    accuarcy = correct / len(of_data_loader.dataset)
    return test_loss, accuarcy, correct

In [102]:
def test_single(model, data_loader):
    device = torch.device("cuda")
    model = model.to(device)
    with torch.no_grad():
        correct = 0
        test_loss = 0
        for data, target in data_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)
            target_view = target.view_as(pred)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(data_loader)
    accuarcy = correct / len(data_loader.dataset)
    return test_loss, accuarcy

In [101]:

first_batch_test = torch.utils.data.Subset(test_dataset, range(0, 64))
exp_test_loader = torch.utils.data.DataLoader(first_batch_test, batch_size=32, shuffle=False)

first_batch_testOF = torch.utils.data.Subset(test_datasetOF, range(0, 64))
exp_test_loaderOF = torch.utils.data.DataLoader(first_batch_testOF, batch_size=32, shuffle=False)


print("model:\t\t(loss,\t\taccuracy)")
print("Image:\t\t",test_single(model, exp_test_loader))
print("Flow:\t\t",test_single(model1, exp_test_loaderOF))
print("Combined:\t",test_combined(model, model1, exp_test_loader, exp_test_loaderOF))

model:		(loss,		accuracy)
Image:		 (3.3590206503868103, 0.484375, 31)
Flow:		 (1.4815059900283813, 0.609375, 39)
Combined:		 (3.8649808168411255, 0.640625, 41)
